# Anatomy of quantum annealing with D-Wave on Amazon Braket

This tutorial notebook dives deep into the __anatomy of quantum annealing__ with D-Wave on Amazon Braket.

First, the concept of quantum annealing as used by D-Wave is introduced to show how it probabilistically finds the (approximate) optimum to some optimization problem. 

The next section introduces the structures of the D-Wave QPUs and the concept of __embedding__. Amazon Braket provides two D-Wave devices, 2000Q and Advantage. The 2000Q device has the __Chimera__ topology, while the Advantage device has the __Pegasus__ topology. Running a problem on a particular D-Wave device requires mapping the original source graph onto the target graph. This mapping is called embedding.

Finally, an example QUBO problem is solved using the QPU to demonstrate the sampling process and a breakdown of the QPU access time. 

## Background: embedding

To solve a real-world problem such as some scheduling problem or a traveling salesman problem with a quantum annealer as provided by D-Wave, in essence two subproblems need to be solved: 
* __Reduction__: First, to map the use case to QUBO (or the equivalent Ising) form written as

$$\mathrm{min} \hspace{0.1cm} y=x^{\intercal}Qx + x^{\intercal}B + c,$$ 

where $x=(x_{1}, x_{2}, \dots)$ is a vector of binary decision variables $x_{i}=0,1$ (whereas Ising problems are formulated in terms of binary Ising variables $z_{i}=-1,1$ with the affine mapping $x_{i}=(z_{i}+1)/2$, i.e., $z_{i}=2x_{i}-1$). Note that QUBO problems can be simplified using $x_{i}^{2}=x_{i}$ for $x_{i}=0,1$. For an excellent overview on how to convert many NP-hard use cases to QUBO form, refer to Ref.[1].
* __Embedding__: Second, the logical problem with connections expressed by the matrix $Q$ has to be mapped to the underlying hardware. In the case of the D-Wave 2000Q device this is a Chimera graph, while for the Advantage device this is a Peganus graph. 

The approach to address the embedding problem is given by the __minor embedding__ technique:
A minor-embedding is a mapping that maps a (logical) graph $G$ to a sub-graph of another (hardware) graph $U$. 
One can solve problems with higher connectivity directly on the sparsely connected chip, by sacrificing physical qubits accordingly to the connectivity of problem, introducing a certain overhead with multiple physical qubits making up one logical variable. For illustration, the following shows an example on how to embed a highly connected graph (with maximum degree 5) to a more sparsely connected graph (with maximum degree 3 only), where (after the embedding) the logical node 1 is encoded by three physical qubits that are strongly interlinked with each other to ensure a common encoding (without chain breaking).   


The minor embedding problem itself is NP-hard: For example, for problems where the number of logical variables is more than 64, there is no guarantee to find an embedding on the D-Wave 2000Q device. The embedding process is normally implemented when a sampler method is called, while one can manually find graph minors, using a method called _Minorminer_ (developed by D-Wave). 


## Imports and setup

In [ ]:
from braket.aws import AwsDevice
from braket.ocean_plugin import BraketSampler, BraketDWaveSampler

In [ ]:
from IPython.display import Image

In [ ]:
import matplotlib.pyplot as plt
# magic word for producing visualizations in notebook
%matplotlib inline
import time
from collections import defaultdict
from itertools import combinations
import math
import networkx as nx
import dwave_networkx as dnx
import minorminer
import dimod
from dimod.binary_quadratic_model import BinaryQuadraticModel
from dwave.system.composites import EmbeddingComposite

In [ ]:
# or choose the D-Wave Advantage device
device = AwsDevice("arn:aws:braket:::device/qpu/d-wave/Advantage_system4")
print('Device:', device)

In [ ]:
# investigate D-Wave device properties
device.properties.provider
device_name = device.name
qubit_count = device.properties.provider.qubitCount
number_couplers = len(device.properties.provider.couplers)
shots_range = device.properties.service.shotsRange
print('Running on {} with {} physical qubits, {} couplers and shots in the range {}.'.format(device_name,
                                                                                             qubit_count,
                                                                                             number_couplers,
                                                                                             shots_range))

## The structures of D-Wave QPUs

D-Wave's 2000Q hardware has a Chimera graph, which has 2048 qubits, consisting of $16 \times 16$ unit cells of $8$ qubits each.
The connectivity within such a unit cell of the Chimera graph is displayed in the following figure (taken from the D-Wave documentation [here](https://docs.dwavesys.com/docs/latest/c_gs_4.html)):  

![chimera_graph.png](attachment:chimera_graph.png)

The Advantage hardware has a Pegasus graph, with in total 5760 qubits. The structure and the connectivity of the Pegasus graph can be described with the following figure (also taken [here](https://docs.dwavesys.com/docs/latest/c_gs_4.html)):

![pegasus_graph.png](attachment:pegasus_graph.png)

In the figure, each colored loop can be regarded as one qubit, which is associated with an odd coupler to form a pair (qubits of the same color). Take the example of qubit 1 (in red): It is internally coupled with 12 qubits (3,4,5,6,7,8) and externally coupled with 2 adjacent qubits (2,9) in the same direction.

__Working graph__: In a D-Wave QPU, the set of qubits and couplers that are available for computation is known as the *working graph*. Because of production imperfections, the yield of a working graph is typically less than the total number of qubits and couplers that are fabricated and physically present in the QPU.

Both the Chimera and the Pegasus graphs are available as a ```networkx``` graph in the package ```dwave_networkx```. 
Using this package, you can visualize some small versions of both graphs [2,3]. 

### The Chimera graph

In [ ]:
# show 1x2 Chimera graph
connectivity_structure = dnx.chimera_graph(1, 2)
dnx.draw_chimera(connectivity_structure)
plt.show()

In [ ]:
# show 2x2 Chimera graph
connectivity_structure = dnx.chimera_graph(2, 2)
dnx.draw_chimera(connectivity_structure)
plt.show()

### The Pegasus graph

In [ ]:
# show the basic Pegasus graph (2 nodes)
connectivity_structure = dnx.pegasus_graph(2)
dnx.draw_pegasus(connectivity_structure)
plt.show()

In [ ]:
# show a larger Pegasus graph (3 nodes)
connectivity_structure = dnx.pegasus_graph(3)
dnx.draw_pegasus(connectivity_structure)
plt.show()

## Finding an embedding  

The next cell generates a fully-connected graph with 9 vertices and $4\times8=32$ nodes. Such fully connected graphs cannot fit onto the sparse graph of the underlying hardware. The following example shows how to use the ```minorminer``` funtion to find an embedding on a Chimera graph.

In [ ]:
# generate a fully connected graph with 9 qubits
G = nx.complete_graph(9)
plt.axis('off') 
nx.draw_networkx(G, with_labels=False)

In [ ]:
# find an embedding on Chimera using minorminer
connectivity_structure = dnx.chimera_graph(2, 2)
embedded_graph = minorminer.find_embedding(G.edges(), connectivity_structure.edges())

# plot this mebedding
dnx.draw_chimera_embedding(connectivity_structure, embedded_graph)
plt.show()

Here, qubits that have the same color correspond to one logical variable in the original problem defined by the $K_{9}$ graph. Qubits combined in such way form a chain. Even though the problem only has $9$ variables (nodes), a large portion of all 32 available physical qubits are used on the toy Chimera graph.

In [ ]:
# Now find and show and embedding using Pegasus
connectivity_structure = dnx.pegasus_graph(3)
embedded_graph = minorminer.find_embedding(G.edges(), connectivity_structure.edges())
dnx.draw_pegasus_embedding(connectivity_structure, embedded_graph)
plt.show()

# The Maximum-Cut Problem

This tutorial solves a small instance of the famous __Maximum-Cut (MaxCut)__ Problem using a D-Wave device on Amazon Braket. The MaxCut problem is one of the most famous NP-hard problems in combinatorial optimization. Given an undirected graph $G(V, E)$ with a vertex set $V$ and an edge set $E$, the Max Cut problem seeks to partition $V$ into two sets such that the number of edges between the two sets (considered to be severed by the cut), is a large as possible. Applications can be found (for example) in clustering problems for marketing purposes or portfolio optimization problems in finance. 

Disclaimer: The code shown in this example has been taken from D-Wave tutorial available online [here]( https://github.com/dwave-examples/maximum-cut), with copyright to D-Wave Systems, Inc., licensed under the Apache License. The purpose of this example is to show how existing code using D-Wave's Ocean tool suite can easily be run on Amazon Braket, with minimal code changes. 

In [ ]:
import json
from braket.aws import AwsDevice
from braket.ocean_plugin import BraketSampler, BraketDWaveSampler

In [ ]:
import matplotlib.pyplot as plt
# magic word for producing visualizations in notebook
%matplotlib inline
import networkx as nx
import dwave_networkx as dnx
from dimod.binary_quadratic_model import BinaryQuadraticModel
from dwave.system.composites import EmbeddingComposite

In [ ]:
# session and device
print('Device:', device)

## SETTING UP THE MAXCUT PROBLEM 

In [ ]:
# helper function to plot graph
def get_graph(graph, pos):
    """
    plot colored graph for given solution
    """
    # positions for all nodes
    # pos = nx.spring_layout(graph)

    # nodes
    nx.draw_networkx_nodes(graph, pos, node_size=700)

    # edges
    nx.draw_networkx_edges(graph, pos)

    # labels
    nx.draw_networkx_labels(graph, pos, font_size=20, font_family='sans-serif')

    # plot the graph
    plt.axis('off')
    #plt.savefig("figures/random_graph.png") # save as png
    plt.show();

In [ ]:
# Copyright 2019 D-Wave Systems, Inc.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# ------ Import necessary packages ----
from collections import defaultdict

# Create empty graph
G = nx.Graph()

# Add edges to the graph (also adds nodes)
# Add the edges as an array of tuples - hint look at max-cut-graph.png for the structure e.g. [(1,2), (1,3 etc etc
G.add_edges_from([(1,2), (1,3), (2,4), (3,4), (3,5), (4,5)])

# plot graph
pos = nx.spring_layout(G)
get_graph(G, pos)

# ------- Set up the QUBO dictionary -------

# Initialize our Q matrix
Q = defaultdict(int)

# Update Q matrix for every edge in the graph
for u, v in G.edges:
    Q[(u,u)]+= -1
    Q[(v,v)]+= -1
    Q[(u,v)]+= 2

#print Q matrix
# Take the time to examine how the Q matrix looks vs what bias we're trying to introduce in the graph.
print('Show Q matrix:', Q)

## SOLVING THE MAXCUT PROBLEM ON DWAVE

In [ ]:
# ------- Run the QUBO problem on the QPU -------
# Set up QPU parameters
chainstrength = 8
numruns = 20

# Run the QUBO on the Braket solver from your config file
# set sampler
sampler = BraketDWaveSampler(device_arn='arn:aws:braket:::device/qpu/d-wave/Advantage_system4')
sampler = EmbeddingComposite(sampler)
response = sampler.sample_qubo(Q, chain_strength=chainstrength, num_reads=numruns)
energies = iter(response.data())

# ------- Print results to user -------
print('-' * 60)
print('{:>15s}{:>15s}{:^15s}{:^15s}'.format('Set 0','Set 1','Energy','Cut Size'))
print('-' * 60)
for line in response:
    S0 = [k for k,v in line.items() if v == 0]
    S1 = [k for k,v in line.items() if v == 1]
    E = next(energies).energy
    print('{:>15s}{:>15s}{:^15s}{:^15s}'.format(str(S0),str(S1),str(E),str(int(-1*E))))

# ------- Display results to user -------
# Grab best result
# Note: "best" result is the result with the lowest energy
# Note2: the look up table (lut) is a dictionary, where the key is the node index
#   and the value is the set label. For example, lut[5] = 1, indicates that
#   node 5 is in set 1 (S1).
lut = response.lowest().first.sample

# Interpret best result in terms of nodes and edges
S0 = [node for node in G.nodes if not lut[node]]
S1 = [node for node in G.nodes if lut[node]]
cut_edges = [(u, v) for u, v in G.edges if lut[u]!=lut[v]]
uncut_edges = [(u, v) for u, v in G.edges if lut[u]==lut[v]]

# Display best result
pos = nx.spring_layout(G)
nx.draw_networkx_nodes(G, pos, nodelist=S0, node_color='r')
nx.draw_networkx_nodes(G, pos, nodelist=S1, node_color='c')
nx.draw_networkx_edges(G, pos, edgelist=cut_edges, style='dashdot', alpha=0.5, width=3)
nx.draw_networkx_edges(G, pos, edgelist=uncut_edges, style='solid', width=3)
nx.draw_networkx_labels(G, pos)

filename = "maxcut_plot.png"
plt.savefig(filename, bbox_inches='tight')
print("\nYour plot is saved to {}".format(filename))

__DISCUSSION__: The optimal cut size for this toy problem is 5. For every edge connecting nodes of different color we score a point (i.e., cut this edge), as displayed above by dashed lines. Here at maximum we can cut five edges. In this toy example we find multiple optimal degenerate solutions, apart from the obvious $\mathbb{Z}_2$ symmetry (that is, the color choice for the two subsets is arbitrary), because there is no preferred coloring for node 5. This problem is equivalent to finding the antiferromagnetic ground state, in the presence of frustration (as present here in the subgraph with nodes 3-4-5). 

**Your feedback matters: A lot!**

<div class="alert alert-block alert-success">Thanks for your participation today. Please fill the <a href="https://survey.immersionday.com/l3LsHGW4R"> survey</a>. It takes less than a minute.</div>